# Tutorial Text Mining

In [0]:
#Para que no muestre los warnings
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [0]:
# Si la línea de abajo tirar error ejecutar esta celda por paquetes no instalados
nltk.download('stopwords')
!pip install gensim
!pip install pyldavis

In [0]:
import pandas as pd
import numpy as np
import nltk 
import re 
from time import time

from nltk.stem.snowball import EnglishStemmer #Para importar método para aplicar stemming
# Para importar el diccionario de stopwords
from nltk.corpus import stopwords
stop = stopwords.words('english')

#LDA
from gensim.corpora import Dictionary, MmCorpus
from gensim.models.ldamodel import LdaModel
import pyLDAvis
import pyLDAvis.gensim


## Dataset Tutorial

El conjunto de datos proviene de uno de los principales sitios de viajes que contiene opiniones de hoteles proporcionadas por los clientes. Este conjunto de datos proporciona opiniones de un solo hotel. 
https://www.kaggle.com/harmanpreet93/hotelreviews

In [0]:

#importar dataset desde github debido a que subirlo directamente a colaboratory es muy lento, 
#este dataset es una muestra del original, contiene 5000 registros
!wget https://raw.githubusercontent.com/dgarridoa/Lab7---dataset/master/hotel-reviews.csv
!mv hotel-reviews.csv hotel-reviews.csv

--2018-11-29 03:44:34--  https://raw.githubusercontent.com/dgarridoa/Lab7---dataset/master/hotel-reviews.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4695168 (4.5M) [text/plain]
Saving to: ‘hotel-reviews.csv’

hotel-reviews.csv   100%[===================>]   4.48M  --.-KB/s    in 0.1s    

2018-11-29 03:44:35 (39.7 MB/s) - ‘hotel-reviews.csv’ saved [4695168/4695168]

mv: 'hotel-reviews.csv' and 'hotel-reviews.csv' are the same file


In [0]:
df = pd.read_csv('hotel-reviews.csv')
#df = df.sample(n=5000, random_state=1).reset_index() #seleccionamos al azar 5000 registros
print('Largo del corpus:', len(df))
df.head()

Largo del corpus: 5000


,Unnamed: 0,index,User_ID,Description,Browser_Used,Device_Used,Is_Response
0,0,1158,id11484,Went for the weekend shopping. This hotel is j...,Firefox,Desktop,happy
1,1,28038,id38364,spent a get-away one night here and loved it. ...,Google Chrome,Tablet,happy
2,2,22473,id32799,I have stayed in Club Quarters in several citi...,Firefox,Desktop,not happy
3,3,1353,id11679,My daughter and I stayed here on a recent trip...,Firefox,Tablet,happy
4,4,9579,id19905,This Sheraton was just adequate and unfortunat...,Edge,Mobile,not happy


In [0]:
def processing_text(text, stemming=True, stopword=True):
    text = re.sub(r'\n', '', text) #eliminamos el símbolo de salto de línea
    text = re.sub(r'\.', ' ', text) #reemplazamos los puntos por un espacio
    text = re.sub(r'[^\w\s]','', text) #elimina los símbolos de puntuación
    text = re.sub(r'[a-zA-Z]+[0-9]+', '', text) #elimina los caracteres que contienen letras y números
    text = re.sub(r'[0-9]+', ' ', text) #elimina los caracteres numéricos
    
    
    tokens = text.split()  #El string es transformado a una lista, el separador por defecto es un espacio
    
    tokens = [i.lower() for i in tokens] #pasar todo a minúsculas
    tokens = [i for i in tokens if len(i) > 1] #eliminar tokens con menos de 2 elementos
    
    #Eliminar stopwords
    if stopword == True:
        tokens = [i for i in tokens if i not in stop]
    #Stemming
    if stemming==True:    
        tokens = [EnglishStemmer().stem(w) for w in tokens] 
        
    
    return tokens

In [0]:
print('Texto original:', df['Description'][1], '\n')
print('Texto sin stopwords:', processing_text(df['Description'][1], stemming=False, stopword=True), '\n')
print('Texto sin stopwords y con stemming:', processing_text(df['Description'][1], stemming=True, stopword=True), '\n')


Texto original: spent a get-away one night here and loved it. great design, fabulous lobby, bar, restaurant, room, comfortable bed, and perfect central location. will always stay here again 

Texto sin stopwords: ['spent', 'getaway', 'one', 'night', 'loved', 'great', 'design', 'fabulous', 'lobby', 'bar', 'restaurant', 'room', 'comfortable', 'bed', 'perfect', 'central', 'location', 'always', 'stay'] 

Texto sin stopwords y con stemming: ['spent', 'getaway', 'one', 'night', 'love', 'great', 'design', 'fabul', 'lobbi', 'bar', 'restaur', 'room', 'comfort', 'bed', 'perfect', 'central', 'locat', 'alway', 'stay'] 



## Comparación del tamaño del vocabulario

In [0]:
t1=time()

corpus = df[df['Description'].isnull()==False].apply(lambda x: processing_text(x['Description'], stemming=False, stopword=False), axis=1)

#Crear la bolsa de palabras (bag of words)
vocabulary = set()
for doc in corpus:
    vocabulary  |=set(doc)

    
    
fdoc = []   
for doc in corpus:
    d = {key: 0 for key in vocabulary}
    for w in doc:
        d[w] = d[w]+1 #d[w]+=1
    fdoc.append(d)
        

t2=time()
print(t2-t1)


21.16262674331665


In [0]:
df_rawclean = pd.DataFrame(fdoc, columns=vocabulary)
print('Dimensión de la base de datos:', df_rawclean.shape)
df_rawclean.head()

Dimensión de la base de datos: (5000, 20520)


,ambiance,daughterinlaw,elements,helpfuli,exposing,layour,miguels,shoes,recommends,tummy,...,gcs,lumps,revonation,toegther,frette,hairgel,reid,res,strangers,cercas
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Palabras más a menos frecuentes en el corpus

In [0]:
df_rawclean.sum().sort_values(ascending=False).head(10)

the      52240
and      27597
to       20188
was      17151
in       13128
of       11005
we       10896
is       10046
for       9726
hotel     9072
dtype: int64

In [0]:
df_rawclean.sum().sort_values(ascending=False).tail(10)

lackadaisical    1
finals           1
minetta          1
buddakan         1
assembled        1
runjobwalk       1
staffthis        1
creativity       1
retrospect       1
cercas           1
dtype: int64

Si nos fijamos las palabras más frecuentes son stopwords

In [0]:
t1=time()
corpus = df[df['Description'].isnull()==False].apply(lambda x: processing_text(x['Description'], stemming=True, stopword=True), axis=1)

#Crear la bolsa de palabras (bag of words)
vocabulary = set()
for doc in corpus:
    vocabulary  |=set(doc)

    
    
fdoc = []   
for doc in corpus:
    d = {key: 0 for key in vocabulary}
    for w in doc:
        d[w] = d[w]+1 #d[w]+=1
    fdoc.append(d)
        
t2=time()
print(t2-t1)

43.85139989852905


In [0]:
df_clean = pd.DataFrame(fdoc)
print('Dimensión de la base de datos:', df_clean.shape)
df_clean.head()

Dimensión de la base de datos: (5000, 14600)


,aa,aaa,aaaaalll,aba,aback,abandon,abat,abb,abbott,abe,...,zonk,zoo,zoom,zoosafari,zumba,zumi,zzip,zzzs,zzzz,übermodern
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [0]:
100*(20520-14600)/20520

28.84990253411306

El vocabulario disminuyó de 20.520 a 14.600, es decir, se redujo el tamaño del vector de características en un 28,85 %, pero aún es muy grande dada la cantidad de documentos. El vocabulario puede reducirse más eliminando palabras muy frecuentes (que aparecen en todos los documentos al menos una vez), también se puede mejorar eliminando las palabras poco frecuentes, etc.


In [0]:
df_clean.sum().sort_values(ascending=False).head(10)

room     10637
hotel    10081
stay      6227
great     3400
staff     3157
locat     2884
would     2862
night     2775
nice      2547
one       2530
dtype: int64

In [0]:
df_clean.sum().sort_values(ascending=False).tail(10)

furnituredesign    1
furthe             1
rage               1
furthermost        1
furthest           1
raffl              1
raffaello          1
rafeal             1
radiss             1
übermodern         1
dtype: int64

In [0]:
print('Palabras que aparecen 5 o más veces en el corpus:', (df_clean.sum().sort_values(ascending=False)>=5).sum())

Palabras que aparecen 5 o más veces en el corpus: 4171


Nos quedamos solo con las palabras del vocabulario que aparecen almenos  5 veces en el corpus

In [0]:
voc_freq = df_clean.sum().reset_index(name='count')
vocabulary = [*voc_freq[voc_freq['count']>=5]['index']]

fdoc = []   
for doc in corpus:
    d = {key: 0 for key in vocabulary}
    for w in doc:
        if w in d.keys():
            d[w] = d[w]+1 #d[w]+=1
    fdoc.append(d)
        
df_tf = pd.DataFrame(fdoc)
print('Dimensión de la base de datos:', df_tf.shape)
df_tf.head()

Dimensión de la base de datos: (5000, 4171)


,aaa,abil,abl,absenc,absolut,absurd,abund,abus,ac,accent,...,youth,youv,yr,yuck,yum,yummi,zaza,zero,zone,zoo
0,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Lo que se ha hecho hasta aquí es usar vector space model, son una serie de pasos para llevar un documento a una representación vectorial, en este caso la representación es en frecuencia (TF), otras representaciones es la representación binaria (1 si la palabra aparece en el documento 0 si no) y TF-IDF (ver Aux 3).

In [0]:
#TF_IDF

df_tf_idf = df_tf.copy()

N = len(df_tf)

for col in [*df_tf.columns]:
    f_t = len(df_tf[col][df_tf[col]>0]) #veces que el término t aparece al menos una vez dentro del corpus
    idf = np.log(N/f_t) #notar que si la palabra aparece en todos los documentos el idf=0 (ln(N/N)=ln(1))
    
    df_tf_idf[col] = df_tf[col]*idf


df_tf_idf.head()   

,aaa,abil,abl,absenc,absolut,absurd,abund,abus,ac,accent,...,youth,youv,yr,yuck,yum,yummi,zaza,zero,zone,zoo
0,0.0,0.0,0.0,0.0,3.203987,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## LDA

más documentación en https://radimrehurek.com/gensim/models/ldamodel.html

In [0]:
#Creamos un nuevo corpus usando como base el vocabulario reducido, es decir, omitiendo las palabras que no estan en el vocabulario

newcorpus = []

for doc in corpus:
    newdoc = []
    for w in doc:
        if w in vocabulary:
            newdoc.append(w)
            
    newcorpus.append(newdoc)

In [0]:
#Creamos el diccionario a partir de los textos procesados en el formato que necesita LDA en gensim
dictionary = Dictionary(newcorpus)

#Creamos el corpus para darle al modelo (segun el formato de esta libreria)
#El corpus contiene una representacion numerica de los textos, un texto es representada por una lista de tuplas
#donde el primer elemento de la tupla es la id de la palabra y el segundo es su frecuencia de aparición en el texto.


corpus = [dictionary.doc2bow(text) for text in newcorpus]

#guardamos el diccionario y el corpus
dictionary.save('dictionary.dict')
MmCorpus.serialize('corpus.mm', corpus)

In [0]:
print('Primer elemento del diccionario o bolsa de palabras:', dictionary[0])
print('Representación del corpus en el formato que requiere la librería: \n', corpus[0:2] )

Primer elemento del diccionario o bolsa de palabras: absolut
Representación del corpus en el formato que requiere la librería: 
 [[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 3), (15, 1), (16, 1), (17, 2), (18, 3), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1), (29, 1), (30, 1), (31, 1), (32, 1), (33, 1), (34, 1)], [(25, 1), (29, 1), (35, 1), (36, 1), (37, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 1), (50, 1), (51, 1)]]


In [0]:
Lda_k_5=LdaModel(corpus=corpus, id2word=dictionary, num_topics=5,  random_state=1) 
Lda_k_5.save('Lda_k_5.model') # guardamos el modelo

In [0]:
#Las 5 palabras más probables de los 5 tópicos
Lda_k_5.show_topics(num_topics=5, num_words=5, log=False, formatted=True)

[(0,
  '0.041*"hotel" + 0.020*"room" + 0.016*"stay" + 0.009*"locat" + 0.008*"would"'),
 (1,
  '0.035*"room" + 0.025*"hotel" + 0.017*"stay" + 0.014*"great" + 0.009*"nice"'),
 (2,
  '0.021*"hotel" + 0.017*"room" + 0.014*"stay" + 0.010*"one" + 0.009*"would"'),
 (3,
  '0.029*"room" + 0.027*"hotel" + 0.014*"stay" + 0.012*"great" + 0.010*"locat"'),
 (4,
  '0.035*"room" + 0.019*"stay" + 0.016*"hotel" + 0.009*"staff" + 0.008*"night"')]

In [0]:
# Mezcla de tópicos que genera el décimo documento del corpus
Lda_k_5.get_document_topics(corpus, per_word_topics=False, minimum_probability=0)[10]

[(0, 0.0062422995),
 (1, 0.006228853),
 (2, 0.0062257997),
 (3, 0.97506195),
 (4, 0.0062410757)]

In [0]:
lda_display = pyLDAvis.gensim.prepare(Lda_k_5, corpus, dictionary, sort_topics=True, R=30)

In [0]:
pyLDAvis.display(lda_display)

#  Laboratorio

## Base de datos

La base de datos del laboratorio corresponde a los papers presentados en la conferencia organizada por el NIPS (Neural Information Processing Systems)  entre los años 1987-2017, el NIPS es la conferencia más prominente en inteligencia artificial.

Descargar en https://www.kaggle.com/benhamner/nips-papers.

En el presente laboratorio se utilizarán las publicaciones de los últimos 5 años (2013-2018) y se hará análisis de tópicos sobre la variable \textbf{abstract} con el objetivo de encontrar las tendencias de los últimos años en inteligencia artificial.

In [0]:
#importar dataset desde github debido a que subirlo directamente a colaboratory es muy lento
!wget https://raw.githubusercontent.com/dgarridoa/Lab7---dataset/master/papers.csv
!mv papers.csv papers.csv

--2018-11-29 03:37:40--  https://raw.githubusercontent.com/dgarridoa/Lab7---dataset/master/papers.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5098233 (4.9M) [text/plain]
Saving to: ‘papers.csv.1’

papers.csv.1        100%[===================>]   4.86M  14.1MB/s    in 0.3s    

2018-11-29 03:37:40 (14.1 MB/s) - ‘papers.csv.1’ saved [5098233/5098233]

mv: 'papers.csv' and 'papers.csv' are the same file


In [0]:
df = pd.read_csv('papers.csv')
df.head()

,Unnamed: 0,id,year,title,event_type,pdf_name,abstract
0,0,1,1987,Self-Organization of Associative Database and ...,NaN,1-self-organization-of-associative-database-an...,Abstract Missing
1,1,10,1987,A Mean Field Theory of Layer IV of Visual Cort...,NaN,10-a-mean-field-theory-of-layer-iv-of-visual-c...,Abstract Missing
2,2,100,1988,Storing Covariance by the Associative Long-Ter...,NaN,100-storing-covariance-by-the-associative-long...,Abstract Missing
3,3,1000,1994,Bayesian Query Construction for Neural Network...,NaN,1000-bayesian-query-construction-for-neural-ne...,Abstract Missing
4,4,1001,1994,"Neural Network Ensembles, Cross Validation, an...",NaN,1001-neural-network-ensembles-cross-validation...,Abstract Missing


In [0]:
df['abstract'][df['year']>=2013].head()

4261    If a piece of information is released from a m...
4262    The adaptive anonymity problem is formalized w...
4263    Tensor completion from incomplete observations...
4265    Motivated by an application in computational b...
4266    Lifted inference algorithms exploit symmetries...
Name: abstract, dtype: object

## 1. Investigación, 2.5 ptos (0.5ptos c/u)




1. ¿Qué es el modelado de tópicos?¿Para qué sirve? De dos ejemplos de uso.
2. Explique LDA, detalle que significa cada unos de los parámetros ($\theta_{d}, \beta_{k}, \alpha, \eta$).
3. Mencione algún uso alternativo de LDA a parte de encontrar tópicos.

Hint: Piense en que tiene un problema de clasificación  con variables estructuradas y una variable no estructurada (texto), entonces, ¿cómo LDA puede ayudar a mejorar el performance del modelo de clasificación?, tenga en cuenta que el tamaño del vocabulario es muy grande como para concatenar la representación vectorial del documento al resto de variables usando vector space model, ya que si lo hace las variables estructuradas (que son pocas y son buenos predictores) son opacadas por la dimensionalidad del otro vector, además  de ser más sensible a sobreajuste y maldición  de la dimensionalidad.

4. Investigue otro modelo de tópicos y expliquelo, comenté en que se diferencia de LDA.
5. Investigue y explique al menos 2 técnicas para determinar el número óptimo de tópicos. ¿Cuál es el problema de usar métricas para escoger el número óptimo de tópicos? 


## 2. Aplicación, 3.5 ptos


2.1 (0.5 ptos) Cree una función para procesar los textos e imprima en pantalla un ejemplo del dataset sin y con procesar. 
Justifique los supuestos realizados. ¿A cuánto se reduce el vocabulario? 


2.2. (0.5 ptos) Postprocessing: análisis de palabras muy frecuentes, poco frecuentes, errores ortográficos, etc. Justifique los supuestos realizados para reducir el vocabulario aún más. ¿A cuánto se reduce el vocabulario? 


2.3. (3.5 ptos) LDA 

2.3.1 (1.5 ptos) Ejecute LDA para k=2, ...10 e interprete los tópicos encontrados. Argumente.

2.3.2 (0.5 ptos) Escoga el número óptimo de tópicos en base a las métricas investigadas y su juicio. 

2.3.3 (0.5 ptos) Analice el tamaño de los tópicos y concluya. 


Bonus (1 pto): Realice el mismo análisis con el otro modelo de tópicos investigado para el número óptimo de tópicos encontrado en la parte 2.3.2
